In [ ]:
import math

def tau_closest_agents(agent, agents, d, tau) -> tuple[list[str], float]:
    """Return the list of tau-closest agents to agent.
    agent: agent's id
    agents: list of agents' id
    d: distance function
    tau: threshold number (usually number of agents in a cluster)

    Return:
        - list of tau-closest agents' id
        - distance to the furthest agent
    """
    return None # TODO: implement


def SmallestAgentBall(agents, d, tau) -> list[str]:
    """Return the set of per_clusterclosest agents to the agent of the smallest ball.
    N: list of agents' id
    d: distance function
    tau: threshold number (usually number of agents in a cluster)
    """
    if len(agents) <= tau:
        return agents[:]
    lst = [] # (agent, its tau-closest agent, distance) #TODO: find a better name
    for agent in agents:
        closest_agents = tau_closest_agents(agent, agents, d, tau) # (list, distance to the furthest)
        lst.append((agent, *closest_agents))
    min_ball = min(lst, key=lambda x: x[2])
    return min_ball[1]


def GreedyCohesiveClustering(agents, d, k) -> list[list[str]]:
    """ Return the k cohesive clusters of agents by metric d. Each cluster is a list of id.
    agents: list of agents' id
    d: distance function
    k: number of clusters to return
    """
    clusters = [] # each cluster is a list of id
    N = agents[:]
    j = 1
    per_cluster = math.ceil(len(agents)/k)
    while N:
        C_j = SmallestAgentBall(N, d, per_cluster)
        clusters.append(C_j)
        for agent in C_j:
            N.remove(agent)
        j += 1
    return clusters
